In [35]:
import boto3
import getpass
import numpy as np
from preconfig import Preconfig

# 1. Upload config files to S3

In [12]:
# TODO: load template file, convert to .cym files, upload to S3
preconfig = Preconfig()
path_to_template = '../templates/vary_compress_rate.cym.tpl'
configs = preconfig.parse(path_to_template,{})
s3_client = boto3.client("s3")
bucket = 'cytosim-working-bucket'
num_repeats = 5
for config in configs:
    job_name = config[:-4]
    print(job_name)
    with open(config, "rb") as buffer:
        for repeat in range(num_repeats):
            config_name = f'{job_name}/config/{job_name}_{repeat}.cym' 
            print(config_name)
            s3_client.put_object(Bucket=bucket, Key=config_name, Body=buffer)
            
    break

vary_compress_rate0000
vary_compress_rate0000/config/vary_compress_rate0000_0.cym
vary_compress_rate0000/config/vary_compress_rate0000_1.cym
vary_compress_rate0000/config/vary_compress_rate0000_2.cym
vary_compress_rate0000/config/vary_compress_rate0000_3.cym
vary_compress_rate0000/config/vary_compress_rate0000_4.cym


# 2a. Specify job definition

In [13]:
job_definition_arn = "job_definition_arn"

# 2b. Create and register job definition

In [14]:
from container_collection.batch.register_batch_job import register_batch_job

In [15]:
job_definition_name = "test-notebook-cytosim"
image = "simularium/cytosim:latest"
vcpus = 1
memory = 7000
bucket_name = "s3://cytosim-working-bucket/"
simulation_name = "varycompressratezero"

In [24]:
account = getpass.getpass()

In [25]:
job_definition = {
    "jobDefinitionName": job_definition_name,
    "type": "container",
    "containerProperties": {
        "image": image,
        "vcpus": vcpus,
        "memory": memory,
        "environment": [
            {"name": "SIMULATION_TYPE", "value": "AWS"},
            {"name": "S3_INPUT_URL", "value": bucket_name},
            {"name": "SIMULATION_NAME", "value": simulation_name},
        ],
        "jobRoleArn": f"arn:aws:iam::{account}:role/BatchJobRole",
    },
}

In [26]:
account 

'108503617402'

In [27]:
job_definition_arn = register_batch_job(job_definition)

In [ ]:
# Multiple Jobs
# num_repeats = 5
# jobs = np.empty((len(configs), num_repeats), dtype=object)
# for index in range(len(configs)):
#     for repeat in range(num_repeats):
#         jobs[index][repeat] = make_batch_job(f"cytosim-varycompressrate{index}repeat{repeat}", 'simularium/cytosim:latest', '108503617402', 'us-west-2', 'karthikv', 1, 7000, 's3://cytosim-working-bucket/')
#         register_batch_job(jobs[index][repeat])



# 3. Submit job

In [30]:
from container_collection.batch.submit_batch_job import submit_batch_job

In [31]:
job_name = "test-jupyter-submit-job-5312023"
user = "karthikv"
queue = "general"
size = 1

In [32]:
job_arns = submit_batch_job(
    name=job_name,
    job_definition_arn=job_definition_arn,
    user=user,
    queue=queue,
    size=size
)

In [34]:
print(job_arns)

['arn:aws:batch:us-west-2:108503617402:job/2de42883-8c81-42f3-acdd-1ebba7c6467b']


In [36]:
# For loop for multiple job submissions here


NameError: name 'make_batch_job' is not defined

job_arns

# 4. Monitor job status

In [ ]:
# TODO: check job status, print progress bar

# 5. Load results